In [ ]:
import logging

import numpy

import cicada.additive
import cicada.communicator

logging.basicConfig(level=logging.INFO)

@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator, modulus=251, precision=4)
    generator = numpy.random.default_rng(seed=1234)
    
    secret_share = protocol.uniform()
    secret = protocol.encoder.decode(protocol.reveal(secret_share))
    log.info(f"Player {communicator.rank} secret: {secret}")
    
    floor_share = protocol.floor(secret_share)
    floor = protocol.encoder.decode(protocol.reveal(floor_share))
    log.info(f"Player {communicator.rank} floor: {floor%1}")

main();

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:51252 from tcp://127.0.0.1:51252.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:51252 from tcp://127.0.0.1:51253.
INFO:cicada.communicator.nng:Player 2 rendezvous with tcp://127.0.0.1:51252 from tcp://127.0.0.1:51254.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 2 communicator ready.
INFO:root:Player 0 secret: -4.5
INFO:root:Player 1 secret: -4.5
INFO:root:Player 2 secret: -4.5
